# Exercise set 2: 


**Learning Objectives:**

After completing this exercise set, you will be able to:

- Perform least squares regression with multiple predictor variables
- Perform variable selection by inspecting correlations.
- Explain the distinction between training and testing data.
- Calculate effects from an experimental design.


**To get the exercise approved, complete the following problems:**

- [2.1(b)](#2.1(b)): To show that you can select variables by using information on correlations.

- [2.2(c)](#2.2(c)) and [2.2(e)](#2.2(e)): To show that you can create a linear model with many variables and test it.

- [2.3(a)](#2.3(a)), [2.3(b)](#2.3(b)), and [2.3(c)](#2.3(c)): To show that you can calculate effects from an experimental design.

## Exercise 2.1 Predicting blood pressure

The file [bloodpress.csv](bloodpress.csv) contains data for 20 individuals with high blood pressure.
Your goal is to create a least squares model for predicting blood pressure (BP) that achieves $R^2 > 0.95$, using a maximum of two predictor variables.

The columns present in the data file are:

| Column | Description                                                              |             Unit |
|:-------|:-------------------------------------------------------------------------|-----------------:|
| BP     | Blood pressure                                                           |             mmHg |
| Age    | Age                                                                      |            years |
| Weight | Weight                                                                   |               kg |
| BSA    | Body surface area                                                        |            m$^2$ |
| DUR    | Duration of hypertension                                                 |            years |
| BHR    | Basal heart rate                                                         | beats per minute |
| Stress | Stress index (score derived from a standardized questionnaire)           |              --- |

### 2.1(a)

Before building your model, explore the relationships between the blood pressure and the other variables to identify potential predictor candidates for a linear regression model. Do this by creating scatter plots of BP against the other variables and by calculating the [correlation coefficients](https://en.wikipedia.org/wiki/Pearson_correlation_coefficient) between BP and the other variables.

**Task:**

1. Calculate the correlation coefficient between BP and each other variable. This can be done with [pandas](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.corr.html):

```python
import pandas as pd
data = pd.read_csv("bloodpress.csv")
data.corr()
```

2. Create scatter plots of BP against each other variable.

Based on your analysis of the correlations and scatter plots, which of the variables seem most promising for predicting BP?

In [ ]:
# Your code here

#### Your answer to question 2.3(a): Which variables could be most predictive of BP?
*Double click here*

### 2.1(b)

**Task:** Construct a least squares model to predict BP. You can use a maximum of two predictor variables, and the model must achieve an R² value of at least 0.95.

In [ ]:
# Your code here

#### Your answer to question 2.1(b): What variables are you using for your prediction
*Double click here*

## Exercise 2.2: Predicting solubility in water

The dataset [solubility.csv](solubility.csv) contains the measured solubility in water (mol/L) for several molecules. Solubility is a critical property in fields such as drug discovery and environmental chemistry, as it influences how molecules interact with biological systems or the environment. 

In addition to the solubility measurements, we have calculated some molecular descriptors (for instance, the weight of the molecules) using the [RDKit](https://www.rdkit.org/) library. You will use the descriptors to build a predictive least squares model for the solubility.

The raw data is in the following format (showing the first two molecules):

| name                           |   measured log(solubility:mol/L) | SMILES              |   MolWt |   HeavyAtomCount |   RingCount |   LogP    |   MaxPartialCharge |   MinPartialCharge |   NOCount |
|:-------------------------------|---------------------------------:|:--------------------|--------:|-----------------:|------------:|----------:|-------------------:|-------------------:|----------:|
| 1,1,1,2-Tetrachloroethane      |                            -2.18 | ClCC(Cl)(Cl)Cl      | 167.85  |                6 |           0 |    2.5954 |           0.203436 |         -0.122063  |         0 |
| 1,1,1-Trichloroethane          |                            -2    | CC(Cl)(Cl)Cl        | 133.405 |                5 |           0 |    2.3765 |           0.187382 |         -0.0840135 |         0 |


The columns are defined as follows:


| **Column** | **Description** |
|:------------|:-----------------|
| name       | The name of the molecule. |
| measured log(solubility:mol/L) | The logarithm of the measured solubility. The logarithm is used since the solubility can span many orders of magnitude. |
| SMILES     | [SMILES](https://en.wikipedia.org/wiki/Simplified_Molecular_Input_Line_Entry_System) (Simplified Molecular Input Line Entry System) - a way to represent molecular structures as text strings. Included if you want to display the molecule. |
| MolWt      | The molecular weight, measured in g/mol. |
| HeavyAtomCount | The number of heavy (non-hydrogen) atoms in the molecule. |
| RingCount  |  The number of rings in the molecule. |
| LogP    | [LogP (octanol-water partition coefficient)](https://en.wikipedia.org/wiki/Octanol-water_partition_coefficient) - a measure of a molecule's lipophilicity (affinity for fatty or oily environments).  A higher LogP indicates greater lipophilicity, while a lower LogP indicates greater hydrophilicity (affinity for water). |
| MaxPartialCharge | The largest charge on any atom in the molecule (atomic units). |
| MinPartialCharge | The smallest charge on any atom in the molecule (atomic units). |
| NOCount          | The number of Nitrogen and Oxygen atoms in the molecule. |


To reiterate, the goal is to predict `measured log(solubility:mol/L)` from `MolWt`, `HeavyAtomCount`, `RingCount`, `LogP`, `MaxPartialCharge`, `MinPartialCharge`, and `NOCount`.


If you want to visualize some of the molecules, you can use the [RDKit](https://www.rdkit.org/docs/GettingStartedInPython.html) library. For example:

In [ ]:
import numpy as np
import pandas as pd

# RDKit imports for working with molecules:
from rdkit import Chem
from rdkit.Chem import rdCoordGen
from rdkit.Chem.Draw import IPythonConsole

# Draw molecules using svg (instead of png) for better quality:
IPythonConsole.ipython_useSVG = True

data = pd.read_csv("solubility.csv")

In [ ]:
index = 33
smiles_str = data["SMILES"][index]
solubility = data["measured log(solubility:mol/L)"][index]
print(f"Showing SMILES string at index {index}: {smiles_str}")
print(f"Solubility = {solubility}")
# Create a molecule from a smiles string:
mol = Chem.MolFromSmiles(smiles_str)
# Generate 2D coordinates for better visualization (this is not needed):
rdCoordGen.AddCoords(mol)
mol

### 2.2(a)

To predict the solubility $y$, we will make a linear model on the form:

\begin{equation}
\begin{split}
y &= b_0 + b_1 \times (\text{MolWt}) + b_2 \times (\text{HeavyAtomCount}) + b_3 \times (\text{RingCount})\\
&+ b_4 \times (\text{LogP}) + b_5 \times (\text{MaxPartialCharge}) + b_6 \times (\text{MinPartialCharge}) + b_7 \times (\text{NOCount}) .
\end{split}
\end{equation}

This can be expressed in matrix form as:

\begin{equation}
\mathbf{y} = \mathbf{X} \mathbf{b}
\end{equation}

where $\mathbf{y}$ is the vector of measured solubility, $\mathbf{X}$ is the design matrix containing the descriptors, and $\mathbf{b}$ is the vector of coefficients ($b_0$, $b_1$, ..., $b_7$).

The code below attempts to solve for the coefficients $\mathbf{b}$ by directly inverting $\mathbf{X}$:

\begin{equation}
\mathbf{b} =\mathbf{X}^{-1}\mathbf{y}
\end{equation}

However, this approach will fail here.

**Task:** Consider the code below, and do the following:

1. Explain why the code adds a column of ones to $\mathbf{X}$.
2. Explain why inverting $\mathbf{X}$ directly will fail.
3. Correct the code and find $\mathbf{b}$ by using the least squares solution:

\begin{equation}
\mathbf{b} = (\mathbf{X}^\top \mathbf{X})^{-1} \mathbf{X}^\top \mathbf{y}
\end{equation}

In [ ]:
import pandas as pd

data = pd.read_csv("solubility.csv")
data

In [ ]:
# Extract the column with solubility:
y = data["measured log(solubility:mol/L)"].to_numpy().reshape(-1, 1)
# The .reshape(-1, 1) is to reshape y into a column vector.
print(f"Shape of y: {y.shape}")
# Select variables:
descriptors_txt = [
    "MolWt",
    "HeavyAtomCount",
    "RingCount",
    "LogP",
    "MaxPartialCharge",
    "MinPartialCharge",
    "NOCount",
]

# Extract the descriptors:
descriptors = data[descriptors_txt].to_numpy()
# Make a column of ones using np.ones:
ones = np.ones((descriptors.shape[0], 1))
# Add a column of ones to form X:
X = np.hstack((ones, descriptors))
print(f"Shape of X: {X.shape}")

In [ ]:
# Invert X **this will fail**:
b = np.linalg.inv(X) @ y

In [ ]:
# Your code here

#### Your answer to question 2.2(a):
*Double click here*

### 2.2(b)

The least squares problem from [2.2(a)](#2.2(a)) can also be solved using [numpy.linalg.lstsq](https://numpy.org/doc/stable/reference/generated/numpy.linalg.lstsq.html) function or the [Moore-Penrose pseudo-inverse](https://en.wikipedia.org/wiki/Moore%E2%80%93Penrose_inverse), denoted as $\mathbf{X}^+$. The pseudo-inverse is a generalization of the matrix inverse and allows us to calculate the least squares solution b as follows:

\begin{equation}
\mathbf{b} = \mathbf{X}^+  \mathbf{y}
\end{equation}

**Task:** Calculate the least squares solution $\mathbf{b}$ using both [numpy.linalg.lstsq](https://numpy.org/doc/stable/reference/generated/numpy.linalg.lstsq.html) and the pseudo-inverse and compare with the solution you found in [2.2(a)](#2.2(a)).

In [ ]:
# Your code here

#### Your answer to question 2.2(b): Do you find the same $\mathbf{b}$?
*Double click here*

### 2.2(c)

Calculating least squares solutions using matrix operations can be cumbersome, especially remembering to add the column of ones for the intercept. Libraries like [scikit-learn](https://scikit-learn.org) provide more convenient tools for this task. Scikit-learn models are typically created and trained using the `.fit` method of the appropriate model class. The fit method takes two main arguments:

* X: The design matrix. Each row of X represents a single sample and each column represents a variable (feature).
* y: The target variable. The values we want to predict from X.


For least squares regression, here is how we can fit a least squares
model with scikit-learn:
```python
from sklearn.linear_model import LinearRegression

model = LinearRegression(fit_intercept=True)
model.fit(X, y)
```

**Task:** Create a least squares model using [LinearRegression](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LinearRegression.html) from [scikit-learn](https://scikit-learn.org). Compare the model's intercept and coefficients with the least squares solution you calculated in previous parts of this exercise.  You can access the intercept and coefficients using:

```python
print(model.intercept_)  # b0
print(model.coef_)  #b1, b2, ..., b7
```

**Hint:** When using scikit-learn's `LinearRegression`, the design matrix X should not include a column of ones.
The intercept is handled automatically by setting `fit_intercept=True` (which is the default).

**Note:** You may have noted that we import from `sklearn`, this is the *import name* of the `scikit-learn` package.

In [ ]:
# Your code here

#### Your answer to question 2.2(c): What coefficients do you find and what is the intercept?
*Double click here*

### 2.2(d)

A primary purpose of building a predictive model is to *predict* properties for *new* samples. We will now evaluate our solubility model's predictive performance by comparing its predictions (ŷ) to the true measured solubilities (y). With scikit-learn, we can use the `.predict(X)` method to generate predictions from a trained model. This method takes in a design matrix X (containing the feature values for the samples we want to predict) as input:

```python
y_hat = model.predict(X)  # Generate predictions
```

**Task:** Evaluate how well your model is predicting the solubility by:
1. Calculating the [coefficient of determination (R²)](https://en.wikipedia.org/wiki/Coefficient_of_determination).
2. Calculating the [mean absolute error (MAE)](https://en.wikipedia.org/wiki/Mean_absolute_error).
3. Creating a scatter plot with the true solubilities (y) on the x-axis plotted against the predicted solubilities (ŷ) on the y-axis. For perfect predictions, all scatter points should fall on the $y=x$ line. 

**Hint:** Both R² and the MAE can be calculated with methods from [sklearn.metrics](https://scikit-learn.org/stable/api/sklearn.metrics.html): [r2_score](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.r2_score.html#sklearn.metrics.r2_score) and [mean_absolute_error](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.mean_absolute_error.html#sklearn.metrics.mean_absolute_error)

In [ ]:
# Your code here

#### Your answer to question 2.2(d): What values do you get for R² and the MAE?
*Double click here*

### 2.2(e)

When we build a predictive model, as you did in [2.2(c)](#2.2(c)), we say that we **train** the model. We refer to the data we used for this process as the **training data** or the **training set**.

In [2.2(d)](#2.2(d)), you evaluated how well your model reproduces the solubility values it was trained on. This, however, does not provide a realistic evaluation of how well the model generalizes to *new, unseen data*. A model could perfectly "memorize" the training data, achieving excellent performance on it, but fail miserably on new samples. This phenomenon, known as **overfitting**, occurs when the model becomes too specific to the training data and loses its ability to generalize.

To assess how well our model is predicting *new* samples and check for overfitting, we must apply it to a set of data that was not part of the training data.
The file [solubility_test.csv](solubility_test.csv) contains data for additional molecules *not* used in training. This data, called the **test set** or **test data** can be used to **test** how well our model is predicting solubilities for new samples.


**Task:** Evaluate how well your model is predicting the test set:

1. Load the data from `solubility_test.csv` into a new design matrix, `X_test`.

2. Use your trained model to predict the solubilities for the molecules in `X_test`.

3. Calculate R² and the MAE using the *test set* predictions and the true values from `solubility_test.csv`.

4. Create a scatter plot of predicted vs. actual solubilities for the *test set*.

5. Compare the test set performance metrics (R², MAE, and the scatter plot) with the training set performance metrics you calculated previously. Note any differences you observe and explain what these differences might suggest about your model's ability to generalize to new data.

In [ ]:
# Your code here

#### Your answer to question 2.2(e): What values do you get for R² and the MAE for the test set? Are they significantly different from the training data?
*Double click here*

### Remarks for exercise 2.2

**Remark 1:** The variables you have used in X have different units and they span different ranges. Some modelling methods are sensitive to these differences. We will later in TKJ4175 scale such variables to a common range to address this.

**Remark 2:** Machine learning libraries like scikit-learn provide a consistent interface for predictive models. Most models, including linear regression, implement the `.fit()` and `.predict()` methods. This means you can easily experiment with more advanced models by largely reusing your existing code - simply replace the model instantiation, for example:

```python
from sklearn.linear_model import Lasso  # Use LASSO regression
model2 = Lasso(fit_intercept=True)
model2.fit(X, y)
y_hat = model2.predict(X)
```

## Exercise 2.3 Analyzing an experimental design

The growth rate of a particular bacterium species depends on the concentration of nutrients such as phosphate ($P$),
sucrose ($S$), and nitrate ($N$). The table below displays the experimental design used to investigate how these three concentrations influence the growth rate.


|$P$  | $S$ | $N$ | $PS$ | $PN$ | $SN$ | $PSN$ | **Growth rate**  |
|:---:|:---:|:---:|:----:|:----:|:----:|:-----:|:----------------:|
| $+$ | $-$ | $-$ | $-$  | $-$  | $+$  | $+$   | $7$              |  
| $-$ | $+$ | $-$ | $-$  | $+$  | $-$  | $+$   | $10$             | 
| $+$ | $-$ | $+$ | $-$  | $+$  | $-$  | $-$   | $8$              | 
| $-$ | $+$ | $+$ | $-$  | $-$  | $+$  | $-$   | $11$             |  
| $-$ | $-$ | $-$ | $+$  | $+$  | $+$  | $-$   | $11$             |
| $+$ | $+$ | $+$ | $+$  | $+$  | $+$  | $+$   | $12$             |
| $+$ | $+$ | $-$ | $+$  | $-$  | $-$  | $-$   | $7$              |
| $-$ | $-$ | $+$ | $+$  | $-$  | $-$  | $+$   | $7$              | 

### 2.3(a)
Use the information in the table and compute the main effects.

In [ ]:
# Your code here

#### Your answer to question 2.3(a): What are the main effects?
*Double click here*

### 2.3(b)
Verify that the columns for the 2-factor and 3-factor interaction effects are correct in table 2 and compute the interaction effects.

In [ ]:
# Your code here

#### Your answer to question 2.3(b): What are the interaction effects?
*Double click here*

### 2.3(c)
What factors and interactions seem to increase the growth rate?

In [ ]:
# Your code here

#### Your answer to question 2.3(c):
*Double click here*

## Your feedback for Exercise 2

1. **Time & Difficulty:**
    * Length (1=too short, 5=too long): 1  2  3  4  5
    * Difficulty (1=too easy, 5=too difficult): 1  2  3  4  5
    * Most challenging part: _________________________

2. **Code Examples:**
    * More or less example code?  More  Less  About Right
    * Areas where more examples would be helpful: _________________________

3. **Errors/Inconsistencies:** Did you encounter any?  Yes  No  If yes, please describe: _________________________

4. **Suggestions:** How could this exercise be improved? _________________________